In [2]:
import pandas as pd
import numpy as np
import warnings
import pymssql
import locale
import glob
import os

warnings.filterwarnings('ignore')
pd.set_option('use_inf_as_na', True)
pd.options.display.max_columns = None
pd.set_option('float_format', '{:.2f}'.format)
locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [3]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Группа цен], [ID Сайта]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'СИ', N'ИФ', N'ПДК', N'ДК', N'ВЫРИЦА', N'ЧАСЫ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [4]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime) 
df = pd.read_excel(latest_file)

In [5]:
df = df[['article', 'published']]

In [7]:
df = df[df['published'] == True]

In [10]:
df.rename(columns={'article': 'ID Сайта'}, inplace=True)

In [13]:
stock = stock[stock['Группа цен'].str.lower().str.contains('шок')]

In [15]:
stock.drop_duplicates(subset=['ID Сайта'], inplace=True)

In [24]:
stock = stock[stock['ID Сайта'] != '']

In [25]:
stock['ID Сайта'] = stock['ID Сайта'].astype(int)

In [26]:
stock = stock.merge(df, how='left', on='ID Сайта')

In [28]:
stock = stock.drop_duplicates(subset=['ID Сайта'])

In [33]:
stock = stock[(stock['published'] == True) & (stock['Группа цен'] != 'РЕМЕШОК')]

In [34]:
stock.to_excel('опубликованные ID шоков.xlsx', index=False)